In [1]:
import pandas as pd
import numpy as np

import os
import datetime

import preprocessing_rocka
import baseline_extraction
import shape_based_distance
import density_estimation
import model

/Users/baconbaker/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.cluster.dbscan_ module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.cluster. Anything that cannot be imported from sklearn.cluster is now part of the private API.
  warnings.warn(message, FutureWarning)


In [37]:
PATH = '../../../../data/train_data/host/'
files = [PATH + f for f in os.listdir(PATH)]
df = pd.concat([pd.read_csv(f) for f in files])

df_inf = pd.read_csv('../kpi_summary_info.data')

In [149]:
def remove_flat(df, df_info):
    flat_kpis = df_info[(df_info.is_flat == True)]['kpi'].unique()
    df = df[~df.name.isin(flat_kpis)]
    return df

def get_timestamps(df, interval):
    curr_time = np.max(df['timestamp'])
    timestamps = [curr_time - x*(interval*1000*60) for x in range(int(60*24/(interval)))]
    return timestamps
    
def split_freq(df, df_info):
    min1_kpis = df_info[(df_info.interval == '1min')]['kpi'].unique()
    min5_kpis = df_info[(df_info.interval == '5min')]['kpi'].unique()
    
    df_1min = df[df.name.isin(min1_kpis)]
    df_5min = df[df.name.isin(min5_kpis)]
    
    return df_1min, df_5min

def handle_intervals(df, interval):
    df.resample(str(interval)+'T').sum()
    print(df.head())
    return df

def stuffing(df, interval):
    out_df = pd.DataFrame(columns=list(df.columns))
    hosts = df.cmdb_id.unique()
    for host in hosts:
        df_h = df[df.cmdb_id == host]
        for name in df_h.name.unique():
            df_hn = df_h[df_h.name == name]
            df_hn = handle_intervals(df_hn, interval)
            df_hn
            

SyntaxError: unexpected EOF while parsing (<ipython-input-149-7236762f2e70>, line 27)

In [148]:
df_x = remove_flat(df, df_inf)
df1, df5 = split_freq(df_x, df_inf)

df1['timestamp'] = df1['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000.))
hosts = df1['cmdb_id'].unique()

    
df1 = df1.set_index('timestamp').sort_values('timestamp')

df5['timestamp'] = df5['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000.))
df5 = df5.set_index('timestamp').sort_values('timestamp')

df5.resample('5T')

print(df5)
df1 = df1[['cmdb_id','name','value']].groupby(['cmdb_id', 'name'])
df5 = df5[['cmdb_id','name','value']].groupby(['cmdb_id','name'])

<ipython-input-148-5530274e49e9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['timestamp'] = df1['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000.))


                              itemid                      name     bomc_id  \
timestamp                                                                    
2020-05-03 18:00:00  999999996432300          Num_of_processes  ZJ-001-076   
2020-05-03 18:00:01  999999996432301            Zombie_Process  ZJ-001-126   
2020-05-03 18:00:02  999999996432302  Num_of_running_processes  ZJ-001-075   
2020-05-03 18:00:02  999999996430202             Shared_memory  ZJ-001-016   
2020-05-03 18:00:03  999999996487803  System_wait_queue_length  ZJ-001-078   
...                              ...                       ...         ...   
2020-05-04 17:59:57         63309297              Sent_packets  ZJ-001-071   
2020-05-04 17:59:58         63309298              Sent_packets  ZJ-001-071   
2020-05-04 17:59:58  999999996482098                Sent_queue  ZJ-001-064   
2020-05-04 17:59:59         63309299              Sent_packets  ZJ-001-071   
2020-05-04 17:59:59  999999996482099            Received_queue  

<ipython-input-148-5530274e49e9>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['timestamp'] = df5['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000.))


In [122]:
minute_kpis = dict(tuple(df1))
minute5_kpis = dict(tuple(df5))
print(minute_kpis.keys(), minute5_kpis.keys())

dict_keys([('db_001', 'ACS'), ('db_001', 'AIOS'), ('db_001', 'AWS'), ('db_001', 'CPU_Used_Pct'), ('db_001', 'CPU_free_pct'), ('db_001', 'Call_Per_Sec'), ('db_001', 'DFParaWrite_Per_Sec'), ('db_001', 'DbTime'), ('db_001', 'Exec_Per_Sec'), ('db_001', 'LFParaWrite_Per_Sec'), ('db_001', 'LFSync_Per_Sec'), ('db_001', 'Logic_Read_Per_Sec'), ('db_001', 'Login_Per_Sec'), ('db_001', 'MEM_Total'), ('db_001', 'MEM_Used'), ('db_001', 'MEM_Used_Pct'), ('db_001', 'MEM_real_util'), ('db_001', 'New_Tbs_Free_Gb'), ('db_001', 'New_Tbs_Used_Pct'), ('db_001', 'Physical_Read_Per_Sec'), ('db_001', 'Proc_Used_Pct'), ('db_001', 'Proc_User_Used_Pct'), ('db_001', 'Redo_Per_Sec'), ('db_001', 'SctRead_Per_Sec'), ('db_001', 'SeqRead_Per_Sec'), ('db_001', 'Sess_Active'), ('db_001', 'Sess_Connect'), ('db_001', 'Session_pct'), ('db_001', 'TPS_Per_Sec'), ('db_001', 'User_Commit'), ('db_001', 'tnsping_result_time'), ('db_002', 'ACS'), ('db_002', 'AIOS'), ('db_002', 'AWS'), ('db_002', 'CPU_Used_Pct'), ('db_002', 'CPU_fr

In [127]:
val_arr = []
val_ind = []
for host, kpi in minute_kpis.keys():
    val_arr.append(minute_kpis[(host, kpi)]['value'].values)
    val_ind.append((host, kpi))

print([len(x) for x in val_arr], val_ind[-2])    

[1440, 1440, 1440, 1326, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1152, 1152, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1327, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1296, 1296, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1328, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1327, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1152, 1152, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1327, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 2016, 2016, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1328, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 144